<a href="https://colab.research.google.com/github/RakeshBabuMacharla/PDS-Assignment-2/blob/main/Pds_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [147]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv("train.csv")

#a) Handle Missing Values







In [141]:
# Check missing values
print(df.isnull().sum())

Unnamed: 0              0
Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  38
New_Price            5032
Price                   0
dtype: int64


In [142]:
# Handle missing values
# Drop rows where 'Price' is missing (critical for prediction)
df.dropna(subset=['Price'], inplace=True)

# 1. Clean 'Engine' column: Remove 'CC' and convert to float
df['Engine'] = df['Engine'].str.replace(' CC', '').astype(float)

# 2. Clean 'Power' column: Remove 'bhp' and handle 'null' values
df['Power'] = df['Power'].str.replace(' bhp', '')
df['Power'] = pd.to_numeric(df['Power'], errors='coerce')  # Convert to numeric, invalid as NaN

# 3. Clean 'Mileage' column: Extract numerical values and standardize unit
def clean_mileage(mileage):
    if isinstance(mileage, str):
        if 'kmpl' in mileage:
            return float(mileage.replace(' kmpl', ''))
        elif 'km/kg' in mileage:
            # Convert km/kg to kmpl (assuming 1 km/kg ≈ 1.4 kmpl for LPG/CNG)
            return float(mileage.replace(' km/kg', '')) * 1.4
    return np.nan  # If not a string or invalid

df['Mileage'] = df['Mileage'].apply(clean_mileage)

# 4. Impute numerical columns with median
numerical_cols = ['Engine', 'Power', 'Mileage']
for col in numerical_cols:
    df[col] = df[col].fillna(df[col].median())

# 5. Impute 'Seats' with mode (discrete value)
df['Seats'] = df['Seats'].fillna(df['Seats'].mode()[0])

# 6. Drop 'New_Price' (too many missing values)
df.drop('New_Price', axis=1, inplace=True)

# 7. Handle categorical columns (if any missing)
categorical_cols = ['Fuel_Type', 'Transmission', 'Owner_Type']
for col in categorical_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

# Verify no missing values remain
print(df.isnull().sum())

Unnamed: 0           0
Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
Price                0
dtype: int64


# b) Clean Units from Columns

In [143]:
def remove_units(column, pattern):
    return pd.to_numeric(column.astype(str).str.replace(pattern, "", regex=True), errors='coerce')

if "Mileage" in df.columns:
    df["Mileage"] = remove_units(df["Mileage"], r" kmpl| km/kg")

if "Engine" in df.columns:
    df["Engine"] = remove_units(df["Engine"], r" CC")

if "Power" in df.columns:
    df["Power"] = remove_units(df["Power"], r" bhp")

if "New_Price" in df.columns:
    df["New_Price"] = df["New_Price"].astype(str).str.replace(" Lakh", "").str.replace(" Cr", "")
    df["New_Price"] = df["New_Price"].apply(lambda x: float(x)*100 if "Cr" in x else float(x) if x.replace('.', '', 1).isdigit() else np.nan)

    print(df[['Mileage', 'Engine', 'Power', 'New_Price']].head())

#c) One-Hot Encode Categorical Features

In [144]:
# 1. Check unique categories
print("Fuel_Type categories:", df["Fuel_Type"].unique())
print("Transmission categories:", df["Transmission"].unique())

# 2. One-hot encode 'Fuel_Type' and 'Transmission'
df_encoded = pd.get_dummies(
    df,
    columns=["Fuel_Type", "Transmission"],
    prefix=["Fuel", "Transmission"],
    drop_first=False, # Keep all categories (optional)
    dtype=int
)
# 3. Display the new columns
print(df_encoded.filter(regex="Fuel_|Transmission_").head())

Fuel_Type categories: ['Diesel' 'Petrol' 'Electric']
Transmission categories: ['Manual' 'Automatic']
   Fuel_Diesel  Fuel_Electric  Fuel_Petrol  Transmission_Automatic  \
0            1              0            0                       0   
1            0              0            1                       0   
2            1              0            0                       0   
3            1              0            0                       1   
4            1              0            0                       0   

   Transmission_Manual  
0                    1  
1                    1  
2                    1  
3                    0  
4                    1  


#d) Feature Engineering

In [145]:
from datetime import datetime
# Calculate current year
current_year = datetime.now().year

# Create new feature: Car Age
df['Car_Age'] = current_year - df['Year']

# Verify the new column
print(df[['Name', 'Year', 'Car_Age']].head())

                               Name  Year  Car_Age
0  Hyundai Creta 1.6 CRDi SX Option  2015       10
1                      Honda Jazz V  2011       14
2                 Maruti Ertiga VDI  2012       13
3   Audi A4 New 2.0 TDI Multitronic  2013       12
4            Nissan Micra Diesel XV  2013       12


#e) Data Operations

In [146]:
# Select sample columns
selected = df[["Name", "Location", "Year", "Car_Age", "Price"]].head()

# Filter rows with Price > 10 Lakh
filtered = df[df["Price"] > 10].head()

# rename
df = df.rename(columns={"Odometer": "Kilometers_Driven"})

# mutate (already done for Car_Age)

# Sorted by Price (descending)
sorted_df = df.sort_values(by="Price", ascending=False).head()

# Group by Location and summarize average Price
grouped = df.groupby("Location")["Price"].mean().reset_index().sort_values(by="Price", ascending=False)



# Display outputs
print("\n--- Selected Columns ---")
print(selected)

print("\n--- Filtered (Price > 10 Lakh) ---")
print(filtered[["Name", "Location", "Price"]])

print("\n--- Top 5 Expensive Cars ---")
print(sorted_df[["Name", "Location", "Price"]])

print("\n--- Average Price by Location ---")
print(grouped)


--- Selected Columns ---
                               Name    Location  Year  Car_Age  Price
0  Hyundai Creta 1.6 CRDi SX Option        Pune  2015       10  12.50
1                      Honda Jazz V     Chennai  2011       14   4.50
2                 Maruti Ertiga VDI     Chennai  2012       13   6.00
3   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013       12  17.74
4            Nissan Micra Diesel XV      Jaipur  2013       12   3.50

--- Filtered (Price > 10 Lakh) ---
                                 Name    Location  Price
0    Hyundai Creta 1.6 CRDi SX Option        Pune  12.50
3     Audi A4 New 2.0 TDI Multitronic  Coimbatore  17.74
5   Toyota Innova Crysta 2.8 GX AT 8S      Mumbai  17.50
11   Land Rover Range Rover 2.2L Pure       Delhi  27.00
12     Land Rover Freelander 2 TD4 SE        Pune  17.50

--- Top 5 Expensive Cars ---
                                             Name    Location   Price
3952  Land Rover Range Rover 3.0 Diesel LWB Vogue   Hyderabad  160.00
5620  